In [2]:
from collections import defaultdict

In [3]:
QC_THRESHOLD = 30
SEQUENCE_LENGTH_THRESHOLD = 500
SEQUENCE_LENGTH_MAX = 820
MATCH_REWARD = 1
MISMATCH_PENALTY = 0.1 #ENTER AS POSITIVE
GAP_OPENING_PENALTY = 1 #ENTER AS POSITIVE
GAP_EXTENSION_PENALTY = 0 #ENTER AS POSITIVE
NHEJ_INDEL_RANGE = 10 #how far to look before cutsite when classifying as NHEJ
NHEJ_INDEL_THRESHOLD = 1 #number of indels in the range before cutsite for classifying as NHEJ
BP_AFTER_CUT = "TTGAGG"
HDR_SNPS = ['G', 'T', 'C', 'A', 'T', 'G']

In [4]:
f = open("yeast sequences.csv", "r")
column_titles = f.readline().strip('\n')
YeastSequences = defaultdict(dict)

while line := f.readline().strip('\n'):
    line_elements = line.split(',')
    
    if len(line_elements[6]) == 4 and int(line_elements[7]) >= QC_THRESHOLD:
        colony = int(line_elements[6])
        colony_type = line_elements[8]
        color = line_elements[9]
        sequence = line_elements[10]
        
        if len(sequence) > SEQUENCE_LENGTH_THRESHOLD and len(sequence) < SEQUENCE_LENGTH_MAX:
            YeastSequences[colony] = {\
                "colony": colony,
                "type": colony_type,
                "color": color,                
                "sequence": sequence.replace(" ", "")
            }

f.close()
len(YeastSequences)

241

In [5]:
wt = open("wt sequence.txt", "r")
wt_sequence = wt.readline().strip('\n')
wt.close()
wt_sequence

'TTGTTGCATGGCTACGAACCGGGTAATACTAAGTGATTGACTCTTGCTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACTTCGTTGTAAAGAATAAGGAAATGATTCCGGAAGCTTTGGAAGTACTGAAGGATCGTCCTTTGTACGCCGAAAAATGGGCACCATTTACTAAAGAATTAGCAGTCATGATTGTGAGATCTGTTAACGGTTTAGTGTTTTCTTACCCAATTGTAGAGACTATCCACAAGGACA'

In [6]:
def MutationIndexCheck(mut_indices, freqs, i, symbol):
    if symbol == 'T' and freqs['T'] == 163: #cut site
        mut_indices[0] = i
    if symbol == 'A' and freqs['A'] == 219: #mut 1
        mut_indices[1] = i
    if symbol == 'A' and freqs['A'] == 218: #mut 2
        mut_indices[2] = i
    if symbol == 'T' and freqs['T'] == 167: #mut 3
        mut_indices[3] = i
    if symbol == 'G' and freqs['G'] == 113: #mut 4
        mut_indices[4] = i
    if symbol == 'A' and freqs['A'] == 195: #mut 5
        mut_indices[5] = i
    if symbol == 'A' and freqs['A'] == 188: #mut 6
        mut_indices[6] = i
    if symbol == 'G' and freqs['G'] == 117: #PAM
        mut_indices[7] = i

In [7]:
def AffineAlignment(match_reward, mismatch_penalty, gap_open_penalty, gap_extension_penalty, v, w):
    n = len(v) + 1
    m = len(w) + 1
    Backtrack = [[" "]*m for i in range(n)]
    DAG = defaultdict(lambda: (-1,-1,-1)) #DAG: (tier, i, j), tiers: lower = 0, middle = 1, upper = 2

    
    #setup three sets of nodes
    middle = [[0]*(m) for i in range(n)]
    middle[0][1], middle[1][0] = -gap_open_penalty, -gap_open_penalty
    for i in range(2,n):
        middle[i][0] = -gap_extension_penalty * (i - 1) - gap_open_penalty
    for j in range(2,m):
        middle[0][j] = -gap_extension_penalty * (j - 1) - gap_open_penalty
        
    lower = [[float('-inf')]*(m) for i in range(n)]
    upper = [[float('-inf')]*(m) for i in range(n)]
    
    #fill out the three matrices using dynamic programming
    for i in range(1, n):
        for j in range(1, m):
            match = -(mismatch_penalty)
            if v[i - 1] == w[j - 1] or w[j - 1] == 'N':
                match = match_reward
            
            lower[i][j] = max(lower[i-1][j] - gap_extension_penalty, middle[i-1][j] - gap_open_penalty)
            upper[i][j] = max(upper[i][j-1] - gap_extension_penalty, middle[i][j-1] - gap_open_penalty)
            middle[i][j] = max(lower[i][j], upper[i][j], middle[i-1][j-1] + match)
            
            if lower[i][j] == lower[i-1][j] - gap_extension_penalty:
                DAG[(0,i,j)] = (0,i-1,j)
            else:
                DAG[(0,i,j)] = (1,i-1,j) 
                
            if upper[i][j] == upper[i][j-1] - gap_extension_penalty:
                DAG[(2,i,j)] = (2,i,j-1)
            else:
                DAG[(2,i,j)] = (1,i,j-1)
                
            if middle[i][j] == lower[i][j]:
                DAG[(1,i,j)] = (0,i,j)
            elif middle[i][j] == upper[i][j]:
                DAG[(1,i,j)] = (2,i,j)
            else:
                DAG[(1,i,j)] = (1,i-1,j-1)
    
    
    #setup for backtracking
    aligned1 = ""
    aligned2 = ""
    curr_node = (-1,-1,-1)
    max_length = 0
    
    
    #determine which of the three matrices has the maximum path
    if lower[n-1][m-1] >= middle[n-1][m-1] and lower[n-1][m-1] >= upper[n-1][m-1]:
        max_length = lower[n-1][m-1]
        curr_node = (0,n-1,m-1)
    elif middle[n-1][m-1] >= lower[n-1][m-1] and middle[n-1][m-1] >= upper[n-1][m-1]:
        max_length = middle[n-1][m-1]
        curr_node = (1,n-1,m-1)
    else:
        max_length = upper[n-1][m-1]
        curr_node = (2,n-1,m-1)
    
    
    #store indices of our mutations: cut site, 1-6, PAM
    freqs = defaultdict(int)
    mut_indices = [-1] * 8
    aligned_index = 0
    
    #backtrack using the DAG
    i,j = n-1,m-1
    while DAG[curr_node] != (-1,-1,-1):
        temp_node = DAG[curr_node]
        
        if temp_node[0] == 0 and curr_node[0] == 0:
            aligned1 = v[i-1] + aligned1
            aligned2 = "-" + aligned2
            freqs[v[i-1]] += 1
            aligned_index += 1
            MutationIndexCheck(mut_indices, freqs, aligned_index, v[i-1])
            i -= 1
        elif temp_node[0] == 1 and curr_node[0] == 0:
            aligned1 = v[i-1] + aligned1
            aligned2 = "-" + aligned2
            freqs[v[i-1]] += 1
            aligned_index += 1
            MutationIndexCheck(mut_indices, freqs, aligned_index, v[i-1])
            i -= 1
            
        elif temp_node[0] == 1 and curr_node[0] == 1:
            aligned1 = v[i-1] + aligned1
            aligned2 = w[j-1] + aligned2
            freqs[v[i-1]] += 1
            aligned_index += 1
            MutationIndexCheck(mut_indices, freqs, aligned_index, v[i-1])
            i -= 1
            j -= 1
            
        elif temp_node[0] == 2 and curr_node[0] == 2:
            aligned1 = "-" + aligned1
            aligned2 = w[j-1] + aligned2
            aligned_index += 1
            j -= 1
        elif temp_node[0] == 1 and curr_node[0] == 2:
            aligned1 = "-" + aligned1
            aligned2 = w[j-1] + aligned2
            aligned_index += 1
            j -= 1
            
        curr_node = temp_node
    
    #fill in leading dashes
    while i != 0:
        aligned1 = v[i-1] + aligned1
        aligned2 = "-" + aligned2
        i -= 1
    while j != 0: 
        aligned1 = "-" + aligned1
        aligned2 = w[j-1] + aligned2
        j -= 1
    
    for i in range(len(mut_indices)):
        mut_indices[i] = len(aligned1) - mut_indices[i]
    
    #return
    return aligned1, aligned2, mut_indices

In [8]:
YeastSequences[2043]["sequence"]

'GTNNNNTTCATTTCGTCAAATGCTGAGCGTTTTATTAGAACTAAATGGACAATATTATCGGAGCATTTGCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACTTCGTTTGTAAAGA'

In [9]:
def ClassifyRepair(wt_substring, sequenced_substring, incorporation_count):
    indel_count = 0
    for i in range(len(wt_substring)):
        if wt_substring[i] == '-' or sequenced_substring[i] == '-':
            indel_count += 1
            
    if indel_count >= NHEJ_INDEL_THRESHOLD:
        return "NHEJ"
    elif incorporation_count == 0:
        return "perfect/no cut"
    return "HDR"

In [10]:
for colony in YeastSequences:
    aligned1, aligned2, mut_indices = AffineAlignment(\
        MATCH_REWARD, MISMATCH_PENALTY, GAP_OPENING_PENALTY, GAP_EXTENSION_PENALTY, \
        wt_sequence, YeastSequences[colony]["sequence"])

    del YeastSequences[colony]["sequence"]
    YeastSequences[colony]["alignment"] = aligned2
    
    hdr_point_mutations = []
    incorporation_count = 0
    for i in range(1,7):
        index = mut_indices[i]
        hdr_point_mutations.append((index, aligned1[index], aligned2[index]))
        if aligned2[index] == HDR_SNPS[i - 1]:
            incorporation_count += 1
    
    cut_pos = mut_indices[0]
    YeastSequences[colony].update({"repair": \
            ClassifyRepair(aligned1[cut_pos - NHEJ_INDEL_RANGE: cut_pos + NHEJ_INDEL_RANGE], \
                           aligned2[cut_pos - NHEJ_INDEL_RANGE: cut_pos + NHEJ_INDEL_RANGE], incorporation_count)})

    YeastSequences[colony].update({"HDR mutations": hdr_point_mutations})
    YeastSequences[colony].update({"incorporation frequency": "{:.2f}".format(float(incorporation_count / 6))})

    YeastSequences[colony].update({"PAM mutation": \
        (mut_indices[7], aligned1[mut_indices[7]: mut_indices[7] + 2], aligned2[mut_indices[7]: mut_indices[7] + 2]) })

    print(YeastSequences[colony])

{'colony': 2042, 'type': 'pML104-gRNA+HDR-A', 'color': 'White', 'alignment': '-----GGANGGAT----------TATT-C---GT---TG-----TGCATGACCTTTTATTA-GA-CTAAATGGACAAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACT-CGTTTGTAAAGA---------------------------------------------------------------------------------------------------------------------------------------------------------------', 'repair': 'perfect/no cut', 'HDR mutations': [(196, 'A', 'A'), (201, 'A', 'A'

{'colony': 2086, 'type': 'pML104-gRNA+HDR-A', 'color': 'White', 'alignment': '-------------------------------AGNNNTTTGCT-TTGGGCTGACTTTTTATTA-GA-CTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACCCTT--TTGTGAAAAGAACAAAAAAACTCTC-TTCCTT----TTT-----TTCT------TCTTTTTTTTTTTTTTTTTTTTT--------TTTTCT------TTTTTTTTTTTTTTTTTTTTTTCTTTCTTCTCTTTTTT-TCTTTTTTTTCCCCTTTTTTTTTTCT-T-----------', 'repair': 'perfect/no cut', 'HDR mutations': [(196, 'A', 'A'), (201, 

{'colony': 2088, 'type': 'pML104-gRNA+HDR-B', 'color': 'Pink/red', 'alignment': 'G-GTTTNNTGGCGGCG---------------------------TTGCTGACCTTTTATTA-GAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTCGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAATCAGCAAACAGACTCAACATTTAGACGGTAATACTAGGTGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACTATGGAAGAGGTGACT--GTTTTCAAAGA-----GAAA---------AA-------------------------------------AAAAA---------------AAA-------------------------------------------------------------------------------', 'repair': 'HDR', 'HDR mutations': [(195, 'A', 'A'), (200, 'A', 'A'), (238, 

{'colony': 2056, 'type': 'pML104-gRNA+HDR-A', 'color': 'White', 'alignment': 'NN--TGACAT--CTTCGCA--GG------CTA--------------------CTTTTATTA-GA-CTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCATCTTGAAGTCTAGGAATTTGGCTTTCATTGGAAGATGTAACTTCTTTGTAAAG----------------------------------------------------------------------------------------------------------------------------------------------------------------', 'repair': 'perfect/no cut', 'HDR mutations': [(195, 'A', 'A'), (200, 'A', 'A'),

{'colony': 2062, 'type': 'pML104-gRNA', 'color': 'Pink/red', 'alignment': 'TTATTT--T---TTCG-------------TAAGCAAT-----CTT------CTATGTAT--AGAGACTAAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAACTTCGTTTGTAAAGA---------------------------------------------------------------------------------------------------------------------------------------------------------------', 'repair': 'perfect/no cut', 'HDR mutations': [(197, 'A', 'A'), (202, 'A', 'A'),

...

{'colony': 2343, 'type': 'pML104-gRNA+HDR-D', 'color': 'Pink/red', 'alignment': 'TN------------------------TACTTCCTGCC-GACTGTAGCGTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTG----------AAACAGACTCAACATTTAGACGGTAATACTAGGTGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGTT---TTTGTT--AAA-AA-AAG---AT--TTCC-----C------AGT---GA-GAATC--CCTTTG----------AATG-------TTGA---AAGAA---------AT--TTG--------------GGTTT--TCCTTCGTCTTGAATCAA--G----GACTTTGGCAT--ACATGGAAAGA', 'repair': 'NHEJ', 'HDR mutations': [(195, 'A', 'A'), (200, 'A', 'A

{'colony': 2307, 'type': 'pML104-gRNA+HDR-B', 'color': 'Pink/red', 'alignment': 'NNNNNNCC---CTCTG--CCGC------CT--GCG-------CGTGCT-------TATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTAATATTTAGGAGGGGGACAATTGGGACGCATGATTGTTGAATCAGCAAACAGAACTCAACATTTAGAACGGTAATACTAGATGGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAAATGACCCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAG---------------AG-----GGAAAGTAGCCCCC---CTTT-----TCC--------C-TCCTTTC-AGGGGGGGAAAGGGG-------------AGAA------------------AGA------AAAGGTA-AGT-TTT----------------------------------', 'repair': 'HDR', 'HDR mutations': [(194, 'A', 'A'), (199, 'A', 'A'), (

{'colony': 2311, 'type': 'pML104-gRNA', 'color': 'White', 'alignment': 'TTG------------------------------------ACTCTTGCTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTA-----------------------------CCGC--GC---------AC------ACCCCCC------CCCC-AAAA--GG------------AAAG---TGGAAGGGAGG---GGGGGTGC-GT---CGCTTTAATAT----------AAATG---------------------', 'repair': 'perfect/no cut', 'HDR mutations': [(194, 'A', 'A'), (199, 'A', 'A'), (237, 

{'colony': 2347, 'type': 'pML104-gRNA+HDR-B', 'color': 'Pink/red', 'alignment': '--GN----------------GGG----ACT-------TGCCTCTGGGGCTGACCTTTTATTAAGAACTAAATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGGTTCTTGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGCATGATTGTTGAATCAGCAAACAGACTCAACATTTAGACGGTAATACTAGGTGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCAAGGACTTTGGCATACGATGGAAGAGGTACCTTTTTTGGTAAGAA-----AAA------------CTT-GGA-------------CGTCCCTTTG-ATGTTGGAAGAGATTTGGG-----TTTTCCA-----TT--C-GTCTTGA-----------GTCAA-GGACTTTGG------CATACG--ATGG-AGAG--T-TC----------', 'repair': 'HDR', 'HDR mutations': [(196, 'A', 'G'), (201, 'A', 'T'),

{'colony': 1867, 'type': 'pML104-gRNA+HDR-B', 'color': 'Pink/red', 'alignment': '--------------------------------------------------------------------------AATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTTGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGCATGATTGTTGAATCAGCAAACAGGCTCAACATTTAGACGGTAATACTAGGTGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGA-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 'repair': 'HDR', 'HDR mutations': [(194, 'A', 'A'), (199, 'A', 'T'), (237, 'T

In [59]:
red_colonies = set()
white_colonies = set()
recombinant_colonies = set()
hdrA_colonies = set()
hdrB_colonies = set()
hdrC_colonies = set()
hdrD_colonies = set()
nhej_repair_colonies = set()
hdr_repair_colonies = set()
no_cut_colonies = set()

HDR_mutation_incorporations = 0
NHEJ_mutation_incorporations = 0
total_mutation_incorporations = 0
hdrA_nhej_incorporations = 0
hdrB_nhej_incorporations = 0
hdrA_hdr_incorporations = 0
hdrB_hdr_incorporations = 0

for colony in YeastSequences:
    if YeastSequences[colony]["color"] == "Pink/red":
        red_colonies.add(colony)
    else:
        white_colonies.add(colony)
        
    if YeastSequences[colony]["type"] == "pML104-gRNA":
        recombinant_colonies.add(colony)
    elif YeastSequences[colony]["type"] == "pML104-gRNA+HDR-A":
        hdrA_colonies.add(colony)
    elif YeastSequences[colony]["type"] == "pML104-gRNA+HDR-B":
        hdrB_colonies.add(colony)
    elif YeastSequences[colony]["type"] == "pML104-gRNA+HDR-C":
        hdrC_colonies.add(colony)
    else:
        hdrD_colonies.add(colony)
        
    if YeastSequences[colony]["repair"] == "NHEJ":
        nhej_repair_colonies.add(colony)
        
        if YeastSequences[colony]["type"] == "pML104-gRNA+HDR-A":
            total_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            hdrA_nhej_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            NHEJ_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
        elif YeastSequences[colony]["type"] == "pML104-gRNA+HDR-B":
            total_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            hdrB_nhej_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            NHEJ_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            
    elif YeastSequences[colony]["repair"] == "HDR":
        hdr_repair_colonies.add(colony)
        
        if YeastSequences[colony]["type"] == "pML104-gRNA+HDR-A":
            total_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            hdrA_hdr_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            HDR_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
        elif YeastSequences[colony]["type"] == "pML104-gRNA+HDR-B":
            total_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            hdrB_hdr_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
            HDR_mutation_incorporations += round(float(YeastSequences[colony]["incorporation frequency"]) * 6)
    else:
        no_cut_colonies.add(colony)

In [54]:
#Incorporation frequency; just HDR-classified colonies
print(HDR_mutation_incorporations)
print((len(hdrA_colonies.intersection(hdr_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(hdr_repair_colonies))))
print(HDR_mutation_incorporations / (len(hdrA_colonies.intersection(hdr_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(hdr_repair_colonies))))

230
66
3.484848484848485


In [53]:
print(NHEJ_mutation_incorporations)
print((len(hdrA_colonies.intersection(nhej_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(nhej_repair_colonies))))
print(NHEJ_mutation_incorporations / (len(hdrA_colonies.intersection(nhej_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(nhej_repair_colonies))))

92
45
2.0444444444444443


In [60]:
print(total_mutation_incorporations)
print(len(hdrA_colonies.intersection(hdr_repair_colonies)) + \
      len(hdrB_colonies.intersection(hdr_repair_colonies)) + \
      len(hdrA_colonies.intersection(nhej_repair_colonies)) + \
      len(hdrB_colonies.intersection(nhej_repair_colonies)))
print(total_mutation_incorporations / ((len(hdrA_colonies.intersection(hdr_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(hdr_repair_colonies)))\
                                       + (len(hdrA_colonies.intersection(nhej_repair_colonies)) + \
                                      len(hdrB_colonies.intersection(nhej_repair_colonies)))))

322
111
2.900900900900901


In [46]:
print(hdrA_nhej_incorporations)
print(len(hdrA_colonies.intersection(nhej_repair_colonies)))
print(hdrA_nhej_incorporations / len(hdrA_colonies.intersection(nhej_repair_colonies)))

89
43
2.0697674418604652


In [47]:
print(hdrB_nhej_incorporations)
print(len(hdrB_colonies.intersection(nhej_repair_colonies)))
print(hdrB_nhej_incorporations / len(hdrB_colonies.intersection(nhej_repair_colonies)))

3
2
1.5


In [55]:
print(hdrA_hdr_incorporations)
print(len(hdrA_colonies.intersection(hdr_repair_colonies)))
print(hdrA_hdr_incorporations / len(hdrA_colonies.intersection(hdr_repair_colonies)))

18
5
3.6


In [56]:
print(hdrB_hdr_incorporations)
print(len(hdrB_colonies.intersection(hdr_repair_colonies)))
print(hdrB_hdr_incorporations / len(hdrB_colonies.intersection(hdr_repair_colonies)))

212
61
3.4754098360655736


In [67]:
point_mutation_counts = [0] * 6
pam_mutation_counts = 0
for colony in YeastSequences:
    if YeastSequences[colony]['type'] == "pML104-gRNA+HDR-A" or YeastSequences[colony]['type'] == "pML104-gRNA+HDR-B":
        if YeastSequences[colony]['type'] == "pML104-gRNA+HDR-B":
            if YeastSequences[colony]['PAM mutation'][2] == 'AT':
                pam_mutation_counts += 1
        for i in range(len(HDR_SNPS)):
            if YeastSequences[colony]['HDR mutations'][i][2] == HDR_SNPS[i]:
                point_mutation_counts[i] += 1
for i in point_mutation_counts:
    print(i / (len(hdrA_colonies) + len(hdrB_colonies)))
print(pam_mutation_counts / len(hdrB_colonies))

0.20945945945945946
0.2905405405405405
0.4797297297297297
0.41216216216216217
0.41216216216216217
0.3716216216216216
0.8857142857142857


In [30]:
for colony in hdrA_colonies.intersection(no_cut_colonies).intersection(red_colonies):
    print(YeastSequences[colony])

{'colony': 1825, 'type': 'pML104-gRNA+HDR-A', 'color': 'Pink/red', 'alignment': 'T---------------------------------------------------------------------GGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGATTCTAGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGTATGATTGTTGAGGCAGCAAACAGGCTCAACATTAAGACGGTAATACTAGATGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAG-TAA-T---------------------------------------------------------------------------------------------------------------------------------------------------------------------------', 'repair': 'perfect/no cut', 'HDR mutations': [(194, 'A', 'A'), (199, 'A', 'A'

In [28]:
for colony in hdrA_colonies.intersection(nhej_repair_colonies):
    if YeastSequences[colony]['HDR mutations'][2][2] == 'C':
        print(YeastSequences[colony])

{'colony': 2052, 'type': 'pML104-gRNA+HDR-A', 'color': 'Pink/red', 'alignment': 'TNNAAN-ATTTCTTC---CCTTTTGGGTA-------G-----GACT----------TTTTATTACAAACTACATGGACAATATTATGGAGCATTTCATGTATAAATTGGTGCGTAAAATCGTTGGATCTCTCTTCTAAGTACATCCTACTATAACAATCAAGAAAAACAAGAAAATCGGACAAAACAATCAAGTATGGGTTCTTGAACAGTTGGTATATTAGGAGGGGGACAATTGGGACGCATGATTGTTG-GGCAGCAAACAGACTCAACATTTAGACGGTAATACTAGGTGCTGAAAATTCTCCTGCCAAACAAATAAGCAACTCCAATGACCACGTTAATGGCTCCTTTTCCAATCCTCTTGATATCGAAAAACTAGCTGAAAAATGTGATGTGCTAACGATTGAGATTGAGCATGTTGATGTTCCTACACTAAAGAATCTTCAAGTAAAACATCCCAAATTAAAAATTTACCCTTCTCCAGAAACAATCAGATTGATACAAGACAAATATATTCAAAAAGAGCATTTAATCAAAAATGGTATAGCAGTTACCCAAAGTGTTCCTGTGGAACAAGCCAGTGAGACGTCCCTATTGAATGTTGGAAGAGATTTGGGTTTTCCATTCGTCTTGAAGTCGAGGACTTTGGCATACGATGGAAGAGGTAATTTT-TTG-AAAGAG--AGGAAA---------AA-------------------------------------AAAAATACGCTCCCT---CT---------------------------TCTCTTA-------------TTCT---------GT------TA------------', 'repair': 'NHEJ', 'HDR mutations': [(198, 'A', 'G'), (203, 'A', 'T'), (24

In [13]:
#white + NHEJ
print(len(white_colonies.intersection(nhej_repair_colonies).intersection(recombinant_colonies)), \
    len(white_colonies.intersection(nhej_repair_colonies).intersection(hdrA_colonies)), \
    len(white_colonies.intersection(nhej_repair_colonies).intersection(hdrB_colonies)), \
    len(white_colonies.intersection(nhej_repair_colonies).intersection(hdrC_colonies)), \
    len(white_colonies.intersection(nhej_repair_colonies).intersection(hdrD_colonies)), \
    len(white_colonies.intersection(nhej_repair_colonies)))

6 6 0 2 0 14


In [14]:
#white + HDR
print(len(white_colonies.intersection(hdr_repair_colonies).intersection(recombinant_colonies)), \
    len(white_colonies.intersection(hdr_repair_colonies).intersection(hdrA_colonies)), \
    len(white_colonies.intersection(hdr_repair_colonies).intersection(hdrB_colonies)), \
    len(white_colonies.intersection(hdr_repair_colonies).intersection(hdrC_colonies)), \
    len(white_colonies.intersection(hdr_repair_colonies).intersection(hdrD_colonies)), \
    len(white_colonies.intersection(hdr_repair_colonies)))

1 0 7 0 0 8


In [15]:
#white + no cut
print(len(white_colonies.intersection(no_cut_colonies).intersection(recombinant_colonies)), \
    len(white_colonies.intersection(no_cut_colonies).intersection(hdrA_colonies)), \
    len(white_colonies.intersection(no_cut_colonies).intersection(hdrB_colonies)), \
    len(white_colonies.intersection(no_cut_colonies).intersection(hdrC_colonies)), \
    len(white_colonies.intersection(no_cut_colonies).intersection(hdrD_colonies)), \
    len(white_colonies.intersection(no_cut_colonies)))

23 28 7 6 10 74


In [16]:
#red + NHEJ
print(len(red_colonies.intersection(nhej_repair_colonies).intersection(recombinant_colonies)), \
    len(red_colonies.intersection(nhej_repair_colonies).intersection(hdrA_colonies)), \
    len(red_colonies.intersection(nhej_repair_colonies).intersection(hdrB_colonies)), \
    len(red_colonies.intersection(nhej_repair_colonies).intersection(hdrC_colonies)), \
    len(red_colonies.intersection(nhej_repair_colonies).intersection(hdrD_colonies)), \
    len(red_colonies.intersection(nhej_repair_colonies)))

21 37 2 11 11 82


In [17]:
#red + HDR
print(len(red_colonies.intersection(hdr_repair_colonies).intersection(recombinant_colonies)), \
    len(red_colonies.intersection(hdr_repair_colonies).intersection(hdrA_colonies)), \
    len(red_colonies.intersection(hdr_repair_colonies).intersection(hdrB_colonies)), \
    len(red_colonies.intersection(hdr_repair_colonies).intersection(hdrC_colonies)), \
    len(red_colonies.intersection(hdr_repair_colonies).intersection(hdrD_colonies)), \
    len(red_colonies.intersection(hdr_repair_colonies)))

0 5 54 0 1 60


In [18]:
#red + no cut
print(len(red_colonies.intersection(no_cut_colonies).intersection(recombinant_colonies)), \
    len(red_colonies.intersection(no_cut_colonies).intersection(hdrA_colonies)), \
    len(red_colonies.intersection(no_cut_colonies).intersection(hdrB_colonies)), \
    len(red_colonies.intersection(no_cut_colonies).intersection(hdrC_colonies)), \
    len(red_colonies.intersection(no_cut_colonies).intersection(hdrD_colonies)), \
    len(red_colonies.intersection(no_cut_colonies)))

1 2 0 0 0 3


In [19]:
#totals
print(len(recombinant_colonies), \
     len(hdrA_colonies), \
     len(hdrB_colonies), \
     len(hdrC_colonies), \
     len(hdrD_colonies), \
     len(YeastSequences))

52 78 70 19 22 241
